In [1]:
import pickle
import boto3
import uuid
import botocore
from botocore.exceptions import ClientError
import os, time, json, time
from datetime import datetime

from misc import load_from_yaml, save_to_yaml
# import s3, iam, lftn, glue, lambdafn as lfn, sns, eventbridge as event
# from lambdafn import build_lambda_package, print_latest_lambda_logs

from dotenv import load_dotenv
load_dotenv(os.getenv("MY_AWS_DIR", "") + "/.env")

from aws_log_extractor import LogManager, LogFilter, LogLevel

from mylogger import CustomLogger
logger = CustomLogger()

In [2]:
ACCOUNT_ID        = os.environ['AWS_ACCOUNT_ID_ROOT']
REGION            = os.environ.get('AWS_DEFAULT_REGION', 'us-east-1')
# VPC_ID            = os.environ['AWS_DEFAULT_VPC']
# SECURITY_GROUP_ID = os.environ['AWS_DEFAULT_SG_ID']
# SUBNET_IDS        = SUBNET_IDS = os.environ["AWS_DEFAULT_SUBNET_IDS"].split(":")
# SUBNET_ID         = SUBNET_IDS[0]

logger.info(f"VPC_ID: {ACCOUNT_ID}")

INFO: 2025-09-16 20:42:40 [2366322474.py:8] VPC_ID: 530976901147


In [3]:
sts_client           = boto3.client('sts')
rds_client           = boto3.client('rds')
iam_client           = boto3.client('iam')
s3_client            = boto3.client('s3')
glue_client          = boto3.client('glue')
lakeformation_client = boto3.client('lakeformation')
ec2_client           = boto3.client('ec2', region_name=REGION)
ec2_resource         = boto3.resource('ec2', region_name=REGION)
dynamodb_client      = boto3.client('dynamodb')
events_client        = boto3.client('events')
lambda_client        = boto3.client('lambda')
sns_client           = boto3.client('sns')
cw_logs_client       = boto3.client('logs')
logs_client = boto3.client("logs")

# Create a CloudWatch client for Logs
logs_client = boto3.client('logs')

#### S3 Bucket Notification Test

In [13]:
# Your bucket name
bucket_name = "datalake-bkt-06012025"  # replace with var.datalake_bkt.bucket
customer_file_sufix = datetime.now().strftime("%Y%m%d%H%M%S")
customer_file_key = f"bronze/customers/customers_{customer_file_sufix}.csv"

In [11]:

# # Put a test object (this should trigger the Lambda)
# response = s3_client.put_object(
#     Bucket=bucket_name, Key=test_key, Body=json.dumps({"test": "notification"})
# )

# Put a test object (this should trigger the Lambda)
response = s3_client.upload_file(
    Bucket=bucket_name, 
    Key=customer_file_key,
    Filename="/Users/am/mydocs/Software_Development/DEDS/aws/glue/data/customers1.csv"
)

print("PutObject response:")
print(response)


PutObject response:
None


In [6]:
response = s3_client.delete_objects(
    Bucket=bucket_name,
    Delete={
        "Objects": [
            {
                "Key": customer_file_key,
            },
        ],
    },
)

In [ ]:
[
    s3_client.delete_object(Bucket=bucket_name, Key=item["Key"])
    # print(item["Key"])
    for item in s3_client.list_objects_v2(Bucket="datalake-bkt-06012025", Prefix="DQE-Results/customers/")[
        "Contents"
    ]
    if item["Key"]
]

[{'ResponseMetadata': {'RequestId': 'WKHEQVDZQA8D4Q45',
   'HostId': 'YPb0uwfH+YqrCqKu+lZhu8OMTc3TtiewrRgVHfo8xbN1BF0o6Y0r6BIG6afYwBNV5q53QfNMGazSmGyKAal3lIZXALuHxUV6',
   'HTTPStatusCode': 204,
   'HTTPHeaders': {'x-amz-id-2': 'YPb0uwfH+YqrCqKu+lZhu8OMTc3TtiewrRgVHfo8xbN1BF0o6Y0r6BIG6afYwBNV5q53QfNMGazSmGyKAal3lIZXALuHxUV6',
    'x-amz-request-id': 'WKHEQVDZQA8D4Q45',
    'date': 'Wed, 17 Sep 2025 02:09:24 GMT',
    'server': 'AmazonS3'},
   'RetryAttempts': 0}},
 {'ResponseMetadata': {'RequestId': 'WKH46V738ECK1TKD',
   'HostId': 'jp6dPfK/Bdo9sw7VGak4AvDZvtVlkWIcUXRcZ95zKTmur0WUK75v1JPVN6+l1L6auaYrB3UQPEsRmutcq+ILhI9gJO6UmkA4',
   'HTTPStatusCode': 204,
   'HTTPHeaders': {'x-amz-id-2': 'jp6dPfK/Bdo9sw7VGak4AvDZvtVlkWIcUXRcZ95zKTmur0WUK75v1JPVN6+l1L6auaYrB3UQPEsRmutcq+ILhI9gJO6UmkA4',
    'x-amz-request-id': 'WKH46V738ECK1TKD',
    'date': 'Wed, 17 Sep 2025 02:09:24 GMT',
    'server': 'AmazonS3'},
   'RetryAttempts': 0}},
 {'ResponseMetadata': {'RequestId': 'WKH5BJFD6YZ124VY',
   'Ho

#### Extract Lambda logs

In [ ]:
LFN_NAME = "s3-customer-crawler-triggerer"
log_group_name = f"/aws/lambda/{LFN_NAME}"

In [ ]:
log_manager = LogManager(region_name="us-east-1")
lambda_extractor = log_manager.get_lambda_extractor()

# end_time = datetime.now(timezone.utc)
# end_time = datetime.now()
# start_time = end_time - timedelta(hours=8)
# request_id = response["ResponseMetadata"]["RequestId"]

# lambda_logs = lambda_extractor.get_logs_by_time_range(
#     function_name=function_name,
#     start_time=start_time,
#     end_time=end_time,
#     # log_filter=LogFilter(filter_pattern="ERROR"),
#     log_filter=LogFilter(),
# )

lambda_logs = lambda_extractor.get_logs_by_strem_limit(function_name=LFN_NAME)

# lambda_logs = lambda_extractor.get_logs_by_request_id(function_name=LFN_NAME, request_id)

print(lambda_logs)

# Print results
for log in lambda_logs:
    # print(f"{log.timestamp}: {log.level} - {log.message}")
    print(log.message)


[]


#### Test Event Rules

In [13]:
response = events_client.put_events(
    Entries=[
        {
            "Source": "my-custom-source",
            "DetailType": "Glue Crawler State Change",
            "Detail": json.dumps(
                {
                    "state": "Succeeded",
                    "crawlerName": "s3-customer-crawler",  # Use the actual crawler name defined in your rule
                }
            ),
            "EventBusName": "default",
        }
    ]
)

print(json.dumps(response, indent=4))


{
    "FailedEntryCount": 0,
    "Entries": [
        {
            "EventId": "6a799faa-13d3-038b-9d80-f08fee652c10"
        }
    ],
    "ResponseMetadata": {
        "RequestId": "8b1aba51-7342-47f3-990e-b1d4e882a484",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "x-amzn-requestid": "8b1aba51-7342-47f3-990e-b1d4e882a484",
            "content-type": "application/x-amz-json-1.1",
            "content-length": "85",
            "date": "Tue, 26 Aug 2025 15:02:03 GMT"
        },
        "RetryAttempts": 0
    }
}


#### Parametarization of the Job [`SUCCESS`]

In [ ]:
ARGS = {
    "--JOB_NAME"           : "s3-customer-processing-job",
    "--S3_BUCKET_DATALAKE" : "datalake-bkt-08242025",
    "--CATALOG_DB_NAME"    : "glue_catalog_db_bronze",
    "--TABLE_NAME"         : "bronze_customers",
    "--TARGET"             : "s3://datalake-bkt-06012025/silver/customers/"
}

In [43]:
# glue.start_glue_job(JOB_NAME, arguments=ARGS)

In [16]:
job_run_id = glue_client.start_job_run(JobName="customer-dq-eval-job")["JobRunId"]

In [9]:
# Initialize log manager
log_manager = LogManager(region_name="us-east-1")

# Get Glue logs by job run ID
glue_extractor = log_manager.get_glue_extractor()

# Example 1: Get logs for specific Glue job run
job_name = "customer-dq-eval-job"
job_run_id = "jr_8c657a4a93dda331343358c34357bfd0ccc1b7f959820f28dd2a6bb651545a28"

glue_logs = glue_extractor.get_logs_by_job_run_id(
    job_name=job_name,
    job_run_id=job_run_id,
    log_filter=LogFilter(log_level=LogLevel.ERROR),
)

In [12]:
glue_logs = glue_extractor.get_logs_by_job_run_id(
    job_name=job_name,
    job_run_id=job_run_id,
    log_filter=LogFilter(log_level=LogLevel.ERROR),
)

In [ ]:
# Print results
for log in glue_logs[:5]:  # Print first 5 logs
    print(f"{log.timestamp}: {log.level} - {log.message}")


#### Data Quality

In [4]:
DETF = load_from_yaml("/Users/am/mydocs/Software_Development/DEDS/terraform_aws/detf/terraform_variables.yaml")

In [5]:
DETF

{'DATALAKE_BKT': 'datalake-bkt-06012025',
 'GLUE_ASSETS_BKT': 'glue-assets-bkt-06012025',
 'GLUE_CATALOG_DB_BRONZE': 'glue_catalog_db_bronze',
 'GLUE_CATALOG_DB_GOLD': 'glue_catalog_db_gold',
 'GLUE_CATALOG_DB_SILVER': 'glue_catalog_db_silver',
 'GLUE_ROLE_ARN': 'arn:aws:iam::530976901147:role/glue-pipeline-role',
 'GLUE_ROLE_NAME': 'glue-pipeline-role',
 'LFN_ROLE_ARN': 'arn:aws:iam::530976901147:role/lfn-pipeline-role',
 'LFN_ROLE_NAME': 'lfn-pipeline-role',
 'TEMP_BKT': 'glue-temp-bkt-06012025'}

In [6]:
CUSTOMER_DQ_RULESET_NAME = "customer-data-quality-ruleset"

In [8]:
CUSTOMER_DQ_REQ_RUN_ID = glue_client.start_data_quality_rule_recommendation_run(
    DataSource={
        "GlueTable": {
            "DatabaseName": DETF["GLUE_CATALOG_DB_BRONZE"],
            "TableName": "bronze_customers",
        }
    },
    Role=DETF["GLUE_ROLE_NAME"],
    NumberOfWorkers=2,
    Timeout=123,
    CreatedRulesetName="customer-data-quality-ruleset",
    # DataQualitySecurityConfiguration='string',
    # ClientToken='string'
)#["RunId"]

In [10]:
CUSTOMER_DQ_REQ_RUN_ID = CUSTOMER_DQ_REQ_RUN_ID["RunId"]

In [11]:
glue_client.get_data_quality_rule_recommendation_run(RunId=CUSTOMER_DQ_REQ_RUN_ID)

{'RunId': 'dqrun-415cae9873250aaae414d1354729ad88e0d2238c',
 'DataSource': {'GlueTable': {'DatabaseName': 'glue_catalog_db_bronze',
   'TableName': 'bronze_customers'}},
 'Role': 'arn:aws:iam::530976901147:role/glue-pipeline-role',
 'NumberOfWorkers': 2,
 'Status': 'RUNNING',
 'StartedOn': datetime.datetime(2025, 9, 16, 20, 47, 29, 747000, tzinfo=tzlocal()),
 'LastModifiedOn': datetime.datetime(2025, 9, 16, 20, 48, 32, 168000, tzinfo=tzlocal()),
 'ExecutionTime': 0,
 'CreatedRulesetName': 'customer-data-quality-ruleset',
 'ResponseMetadata': {'RequestId': '004ada9a-74c5-4df8-8296-e0f4346ec9df',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 17 Sep 2025 01:48:36 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '411',
   'connection': 'keep-alive',
   'x-amzn-requestid': '004ada9a-74c5-4df8-8296-e0f4346ec9df',
   'cache-control': 'no-cache'},
  'RetryAttempts': 0}}

In [ ]:
# Start DQ Ruleset Evaluation
CUSTOMER_DQ_EVAL_RUN_ID = glue_client.start_data_quality_ruleset_evaluation_run(
    DataSource={
        "GlueTable": {
            "DatabaseName": DETF["GLUE_CATALOG_DB_BRONZE"],
            "TableName": "bronze_customers",
        }
    },
    AdditionalRunOptions={
        "CloudWatchMetricsEnabled": True,
        # "ResultsS3Prefix": DETF["CUSTOMER_DQ_EVAL_RESULTS_S3_PATH"],
        "ResultsS3Prefix": "s3://datalake-bkt-06012025/DQE-Results/customers/",
        "CompositeRuleEvaluationMethod": "COLUMN",  # "COLUMN" | "ROW",
    },
    RulesetNames=[CUSTOMER_DQ_RULESET_NAME],
    Role=DETF["GLUE_ROLE_ARN"],  # <== Add this
)["RunId"]

In [ ]:
glue_client.get_data_quality_ruleset_evaluation_run(RunId=CUSTOMER_DQ_EVAL_RUN_ID)